# Validation neutralization assays versus `polyclonal` fits
Compare actual measured neutralization values for specific mutants to the `polyclonal` fits.

Import Python modules:

In [1]:
import os
import pickle

import altair as alt

import pandas as pd
import numpy as np

import yaml

from scipy import stats

import warnings
warnings.simplefilter("ignore")

palette = ['#999999', '#0072B2',  '#E69F00', '#F0E442', '#009E73','#56B4E9', "#D55E00", "#CC79A7"] 

extended_palette = ['#999999', '#0072B2',  '#E69F00', '#F0E442', '#009E73','#56B4E9', "#D55E00", "#CC79A7", '#9F0162'] 

long_palette = ['#999999', '#9F0162', '#009F81', '#FF5AAF', '#8400CD', '#008DF9', '#00C2F9', '#FFB2FD', '#A40122', '#E20134', '#FF6E3A', '#FFC33B', '#00FCCF']

figure_palette = ['#999999', '#0072B2', '#F0E442', '#E69F00', '#009E73','#56B4E9', "#D55E00", "#CC79A7", '#9F0162','#8400CD']

Read configuration and validation assay measurements:

In [2]:
with open("config.yaml") as f:
    config = yaml.safe_load(f)
    
validation_ic50s = pd.read_csv('data/V3_validation_ICs.csv', na_filter=None)

Now get the predictions by the averaged `polyclonal` model fits:

In [3]:
validation_vs_prediction = []
for virus, virus_df in validation_ic50s.groupby("virus"):
    if virus == 'TRO11':
        virus_data_path = 'results/antibody_escape/averages/'
    elif virus == 'BF520':
        virus_data_path = '../HIV_Envelope_BF520_DMS/results/antibody_escape/averages/'
    for antibody, antibody_df in virus_df.groupby("antibody"):
        with open(os.path.join(virus_data_path, f"{antibody}_polyclonal_model.pickle"), "rb") as f:
            model = pickle.load(f)
        df = model.mut_icXX_df_w_model_values(x=.5, icXX_col='IC50', log_fold_change_icXX_col='log_fold_change_IC50')
        df['virus'] = virus
        df['antibody'] = antibody
        validation_vs_prediction.append(df)
    
validation_vs_prediction = pd.concat(validation_vs_prediction, ignore_index=True)

In [4]:
validation_vs_prediction['aa_substitutions'] = validation_vs_prediction['wildtype'] + validation_vs_prediction['site'] + validation_vs_prediction['mutant']
mutations = validation_ic50s['aa_substitutions'].unique()
plot_data = validation_ic50s.merge(
    (validation_vs_prediction), how='left', on=['aa_substitutions', 'virus', 'antibody'],
)
plot_data.loc[plot_data['aa_substitutions']=='', 'log_fold_change_IC50 mean'] = 0
plot_data.query('times_seen>=2').dropna(subset=['log_fold_change_IC50 mean'])

,antibody,virus,aa_substitutions,measured IC50,Env region,site,wildtype,mutant,log_fold_change_IC50 mean,log_fold_change_IC50 median,...,B-231011-rescue_6_ultra-007-1,A-230906-rescue_5-10-1074-1,B-231011-rescue_6_ultra-10-1074-1,B-231011-rescue_6_ultra-10-1074-2,B-241206-rescue_8-BG18-1,B-241206-rescue_8-BG18-1-dup,A-240223-rescue_6-PGT121-1,B-241206-rescue_8-PGT121-1,A-240223-rescue_6-PGT128-1,B-241206-rescue_8-PGT128-1
1,007,TRO11,N156K,25.000,V1 loop,156,N,K,2.164151,2.164151,...,2.864222,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,007,TRO11,N156R,14.193,V1 loop,156,N,R,2.751599,2.751599,...,3.242581,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,007,TRO11,T189G,0.060,V2 loop,189,T,G,1.284836,1.284836,...,0.680613,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,007,TRO11,T297E,0.002,Site 297,297,T,E,0.345887,0.345887,...,0.636387,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,007,TRO11,T303G,25.000,V3 loop,303,T,G,3.035439,3.035439,...,4.004084,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110,BG18,TRO11,N332V,25.000,N332 glycan,332,N,V,5.764317,5.764317,...,NaN,NaN,NaN,NaN,5.764317,5.764317,NaN,NaN,NaN,NaN
111,BG18,TRO11,S334Q,25.000,N332 glycan,334,S,Q,10.851531,10.851531,...,NaN,NaN,NaN,NaN,10.851531,10.851531,NaN,NaN,NaN,NaN
112,BG18,TRO11,S334R,6.167,N332 glycan,334,S,R,10.780162,10.780162,...,NaN,NaN,NaN,NaN,10.780162,10.780162,NaN,NaN,NaN,NaN
113,BG18,TRO11,S375W,0.014,Site 375,375,S,W,-0.000004,-0.000004,...,NaN,NaN,NaN,NaN,-0.000004,-0.000004,NaN,NaN,NaN,NaN


In [6]:
fold_changes = (
    plot_data
    .rename(columns={"log_fold_change_IC50 mean": "predicted log fold change IC50"})
    [["antibody",
      "virus",
      "aa_substitutions", 
      "measured IC50",
      "predicted log fold change IC50", 
      "times_seen", 
      "n_models",
      "Env region"]]
    .merge(
        plot_data
        .rename(columns={"log_fold_change_IC50 mean": "predicted log fold change IC50"})
        .query("aa_substitutions == ''")
        [["antibody", "virus", "measured IC50", "predicted log fold change IC50"]],
        on=["antibody", "virus"],
        how="left",
        suffixes=[" mutant", " unmutated"],
    )
    .assign(
        measured_fold_change=lambda x: x["measured IC50 mutant"] / x["measured IC50 unmutated"],
    )
)
fold_changes = fold_changes.dropna(subset=['predicted log fold change IC50 mutant'])

In [12]:
#display(plot_data)
for virus in fold_changes['virus'].unique():
    for antibody in fold_changes['antibody'].unique():
        sub_plot_data = fold_changes.query('virus==@virus').query('antibody==@antibody').query('times_seen>=2').copy()
        sub_plot_data['aa_substitutions'] = [f'wildtype {virus}' if x is '' else x for x in sub_plot_data['aa_substitutions']]
        sub_plot_data['log measured_fold_change'] = np.log(sub_plot_data['measured_fold_change'])
        fold_change_chart = (
            alt.Chart(sub_plot_data.query('virus==@virus').query('antibody==@antibody'))
            .encode(
                x=alt.X(
                    "log measured_fold_change",
                    title=["Traditional neutralization assay", "measured log fold change IC50"],
                    scale=alt.Scale(#type="log", 
                                        nice=False,
                                       domain=(sub_plot_data["log measured_fold_change"].min(), 
                                           sub_plot_data["log measured_fold_change"].max()*1.25)),
                       ),
                y=alt.Y(
                    "predicted log fold change IC50 mutant",
                    title=["Deep mutational scanning", "measured log fold change IC50"],
                    scale=alt.Scale(#type="log", 
                                    nice=False,
                                   domain=(sub_plot_data["predicted log fold change IC50 mutant"].min(), 
                                           sub_plot_data["predicted log fold change IC50 mutant"].max()*1.25)),
                ),
                #facet=alt.Facet("antibody", columns=4, title=None),
                color=alt.Color("Env region", 
                                #title="Amino acid substitutions", 
                                scale=alt.Scale(range=figure_palette[1:]),
                                sort=[
                                    'wildtype TRO11',
                                ]
                               ),
                tooltip=[
                    alt.Tooltip(c, format=".3g") if sub_plot_data[c].dtype == float
                    else c
                    for c in sub_plot_data.columns.tolist()
                ],
            )
            .mark_circle(filled=True, size=100, opacity=1)
            #.configure_axis(grid=False)
            #.resolve_scale(y="independent", x="independent")
            .properties(width=150, height=150)
        )
        
        antibody_df = fold_changes.query("antibody==@antibody").query('virus==@virus')
        antibody_df = antibody_df[~antibody_df['aa_substitutions'].str.contains(" ")]
        display(antibody_df)
        print(f"{antibody}:")
        slope, intercept, r_value, p_value, std_err = stats.linregress(
            antibody_df["predicted log fold change IC50 mutant"].astype(float),
            np.log(antibody_df["measured_fold_change"].astype(float)))
        print(f"Predicted fold change correlation (R^2): {round(r_value**2,3)}")
        
        line = alt.Chart(pd.DataFrame({'log measured_fold_change': [sub_plot_data["log measured_fold_change"].max()]})).mark_rule(strokeDash=[8,8]).encode(x='log measured_fold_change')
        (fold_change_chart + line).configure_axis(grid=False).display()

,antibody,virus,aa_substitutions,measured IC50 mutant,predicted log fold change IC50 mutant,times_seen,n_models,Env region,measured IC50 unmutated,predicted log fold change IC50 unmutated,measured_fold_change
0,007,TRO11,,0.009,1.000000,NaN,NaN,,0.009,1.0,1.000000
1,007,TRO11,N156K,25.000,2.164151,11.541667,2.0,V1 loop,0.009,1.0,2777.777778
2,007,TRO11,N156R,14.193,2.751599,5.875000,2.0,V1 loop,0.009,1.0,1577.000000
3,007,TRO11,T189G,0.060,1.284836,4.666667,2.0,V2 loop,0.009,1.0,6.666667
4,007,TRO11,T297E,0.002,0.345887,2.500000,2.0,Site 297,0.009,1.0,0.222222
7,007,TRO11,T303G,25.000,3.035439,3.125000,2.0,V3 loop,0.009,1.0,2777.777778
8,007,TRO11,T303K,25.000,2.797173,9.041667,2.0,V3 loop,0.009,1.0,2777.777778
9,007,TRO11,D322K,25.000,2.274268,7.125000,2.0,V3 loop,0.009,1.0,2777.777778
10,007,TRO11,G324D,0.030,0.244195,10.250000,2.0,V3 loop,0.009,1.0,3.333333
11,007,TRO11,G324V,25.000,1.750852,3.958333,2.0,V3 loop,0.009,1.0,2777.777778


007:
Predicted fold change correlation (R^2): 0.617


alt.LayerChart(...)

,antibody,virus,aa_substitutions,measured IC50 mutant,predicted log fold change IC50 mutant,times_seen,n_models,Env region,measured IC50 unmutated,predicted log fold change IC50 unmutated,measured_fold_change
23,10-1074,TRO11,,0.020,1.000000,NaN,NaN,,0.02,1.0,1.00
24,10-1074,TRO11,N156K,0.038,0.171794,12.166667,3.0,V1 loop,0.02,1.0,1.90
25,10-1074,TRO11,N156R,0.020,0.102984,4.500000,3.0,V1 loop,0.02,1.0,1.00
26,10-1074,TRO11,T189G,0.020,-0.093938,4.111111,3.0,V2 loop,0.02,1.0,1.00
27,10-1074,TRO11,T297E,0.014,0.074094,2.000000,3.0,Site 297,0.02,1.0,0.70
30,10-1074,TRO11,T303G,0.016,-0.022903,2.750000,3.0,V3 loop,0.02,1.0,0.80
31,10-1074,TRO11,T303K,0.015,-0.100386,8.000000,3.0,V3 loop,0.02,1.0,0.75
32,10-1074,TRO11,D322K,0.034,-0.229500,7.416667,3.0,V3 loop,0.02,1.0,1.70
33,10-1074,TRO11,G324D,0.035,-0.184904,9.083333,3.0,V3 loop,0.02,1.0,1.75
34,10-1074,TRO11,G324V,0.001,0.462794,3.750000,3.0,V3 loop,0.02,1.0,0.05


10-1074:
Predicted fold change correlation (R^2): 0.727


...

PGT128:
Predicted fold change correlation (R^2): 0.327


alt.LayerChart(...)

,antibody,virus,aa_substitutions,measured IC50 mutant,predicted log fold change IC50 mutant,times_seen,n_models,Env region,measured IC50 unmutated,predicted log fold change IC50 unmutated,measured_fold_change
92,BG18,TRO11,,0.001,1.000000,NaN,NaN,,0.001,1.0,1.0
93,BG18,TRO11,N156K,0.008,0.028445,16.0,2.0,V1 loop,0.001,1.0,8.0
94,BG18,TRO11,N156R,0.003,0.002307,4.0,2.0,V1 loop,0.001,1.0,3.0
95,BG18,TRO11,T189G,0.002,-0.005168,5.0,2.0,V2 loop,0.001,1.0,2.0
96,BG18,TRO11,T297E,0.001,-0.002313,1.0,2.0,Site 297,0.001,1.0,1.0
99,BG18,TRO11,T303G,0.003,0.004045,3.0,2.0,V3 loop,0.001,1.0,3.0
100,BG18,TRO11,T303K,0.002,-0.006644,10.0,2.0,V3 loop,0.001,1.0,2.0
101,BG18,TRO11,D322K,0.005,-0.000642,7.0,2.0,V3 loop,0.001,1.0,5.0
102,BG18,TRO11,G324D,0.006,0.008777,8.5,2.0,V3 loop,0.001,1.0,6.0
103,BG18,TRO11,G324V,0.001,0.038159,5.5,2.0,V3 loop,0.001,1.0,1.0


BG18:
Predicted fold change correlation (R^2): 0.628


alt.LayerChart(...)